In [1]:
import pandas as pd
import numpy as np

import csv
import json

import re

import itertools

import matplotlib.pyplot as plt
import mpld3

Simple rating calculator (doesn't join any databases) 

by Bobby Mills and Arseny Khakhalin

In [2]:
def fixName(name):
    res = re.sub(' ', '_', name)
    res = re.sub(',_', '_', res)
    res = re.sub('_\d+', '', res) #remove _number from lands
    res = re.sub('_\([a-zA-Z]\)', '', res) #remove guildgate types
    res = res.lower()
    return res

In [4]:
# Change the data file below to work with the draftsim data you want

draftData = pd.read_csv('../2018-10-05 GRN Draft Data 1.csv', 
                       names = ['format', 'human', 'bot1', 'bot2', 'bot3', 'bot4', 'bot5', 'bot6', 'bot7'])
#draftData = pd.read_csv('05-31-18 Rows 500000-600000.csv', 
#                       names = ['format', 'human', 'bot1', 'bot2', 'bot3', 'bot4', 'bot5', 'bot6', 'bot7'])
#draftData = pd.read_csv('../2018-08-23 m19 drafts round 2.csv', 
#                      names = ['format', 'human', 'bot1', 'bot2', 'bot3', 'bot4', 'bot5', 'bot6', 'bot7'])
#draftData = pd.read_csv('2018-04-16 Dominiaria initial data-2.csv', 
#                        names = ['format', 'human', 'bot1', 'bot2', 'bot3', 'bot4', 'bot5', 'bot6', 'bot7'])
draftData.head()

,format,human,bot1,bot2,bot3,bot4,bot5,bot6,bot7
1901310,GRN,"Tajic,_Legion's_Edge,Goblin_Cratermaker,Vernad...","Niv-Mizzet,_Parun,Capture_Sphere,Watcher_in_th...","Status_Statue,Gateway_Plaza,Deadly_Visit,Under...","Lazav,_the_Multifarious,Selective_Snare,Citywa...","Midnight_Reaper,Healer's_Hawk,Collar_the_Culpr...","Lava_Coil,Luminous_Bonds,Rubblebelt_Boar,Hunte...","Luminous_Bonds,Etrata,_the_Silencer,Loxodon_Re...","Find_Finality,Severed_Strands,Douser_of_Lights..."
1901311,GRN,"Pelt_Collector,Gatekeeper_Gargoyle,Murmuring_M...","Discovery_Dispersal,Parhelion_Patrol,Blood_Ope...","Ritual_of_Soot,Hired_Poisoner,Ledev_Guardian,C...","Legion_Warboss,Justice_Strike,Cosmotronic_Wave...","Pelt_Collector,Dead_Weight,Prey_Upon,Sprouting...","Vraska,_Golgari_Queen,Siege_Wurm,Pitiless_Gorg...","Assassin's_Trophy,Hired_Poisoner,Rhizome_Lurch...","Integrity_Intervention,Luminous_Bonds,Invert_I..."
1901312,GRN,"Venerated_Loxodon,Pack's_Favor,Demotion,Ledev_...","Aurelia,_Exemplar_of_Justice,Lava_Coil,Goblin_...","Demotion,Prey_Upon,Dead_Weight,Siege_Wurm,Gate...","Vraska,_Golgari_Queen,Prey_Upon,Gatekeeper_Gar...","Watcher_in_the_Mist,Dead_Weight,Healer's_Hawk,...","Status_Statue,Spinal_Centipede,Burglar_Rat,Moo...","Luminous_Bonds,Inescapable_Blaze,Loxodon_Resto...","Crackling_Drake,Erratic_Cyclops,Fresh-Faced_Re..."
1901313,GRN,"Conclave_Guildmage,League_Guildmage,Flight_of_...","Affectionate_Indrik,Ritual_of_Soot,Severed_Str...","Capture_Sphere,Invert_Invent,Rampaging_Monumen...","Luminous_Bonds,Arboretum_Elemental,Severed_Str...","Lava_Coil,Boros_Challenger,Ornery_Goblin,Direc...","Hunted_Witness,Luminous_Bonds,Plaguecrafter,Bu...","Conclave_Tribunal,Rosemane_Centaur,Rosemane_Ce...","Quasiduplicate,Capture_Sphere,Piston-Fist_Cycl..."
1901315,GRN,"Boros_Challenger,Boros_Guildgate_2,Inspiring_U...","Ritual_of_Soot,Assure_Assemble,Dead_Weight,Col...","Legion_Warboss,Selective_Snare,Goblin_Craterma...","Luminous_Bonds,Demotion,Selesnya_Guildgate_2,L...","March_of_the_Multitudes,Sunhome_Stalwart,Ledev...","Dead_Weight,Flower_Flourish,Prey_Upon,Siege_Wu...","Tajic,_Legion's_Edge,Fire_Urchin,Direct_Curren...","Nightveil_Predator,Unexplained_Disappearance,G..."


In [16]:
# Pick order variable
cardpicks = {}

In [15]:
cardpicks

{"tajic_legion's_edge": 1}

In [17]:
count = 0
player = 'human' # normally should be: human, but can also be bot1 or another bot
for draft in draftData[player]:
    draft = fixName(draft)
    draft = draft.lower()
    draft = draft.split(',')
    count = count+1    
    for i in range(14):
        try:
            if draft[i] in cardpicks:
                cardpicks[draft[i]].append(i+1)
            else:
                cardpicks[draft[i]] = [i+1]       
        except KeyError as e:
            print(draft)
            print(count)
            raise

In [19]:
for card in cardpicks:
    if cardpicks[card] == []:
        cardpicks[card].append(15)

In [20]:
cardpicksdf = pd.DataFrame({
        'avg' : [np.mean(cardpicks[card]) for card in cardpicks],
        'var' : [np.var(cardpicks[card]) for card in cardpicks],
        'count' : [len(cardpicks[card]) for card in cardpicks],
    }, list(cardpicks.keys()))

In [25]:
cardpicksdf = cardpicksdf.sort_values(by=['avg'], ascending=True)

In [26]:
cardpicksdf.to_csv('../Jupyter/simple_rating.csv', index_label="name")

In [27]:
cardpicksdf.head(20)

,avg,var,count
light_of_the_legion,1.000000,0.000000,2027
doom_whisperer,1.000000,0.000000,1252
dream_eater,1.000000,0.000000,1245
aurelia_exemplar_of_justice,1.000000,0.000000,1285
price_of_fame,1.057047,0.055284,2682
status_statue,1.058041,0.057091,1654
find_finality,1.076259,0.071882,1390
dawn_of_hope,1.093120,0.092788,1439
nullhide_ferox,1.112689,0.111354,1056
conclave_tribunal,1.114830,0.108153,2151
